# FPUS23 — YOLO Optimized (n/s/m, balanced) — v2

- Trains YOLO11 n/s/m on class-balanced FPUS23 YOLO dataset.
- Saves checkpoints every epoch to Drive and auto-resumes.
- Uses medical augment profile and cosine LR.


In [ ]:
%pip -q install -U ultralytics >/dev/null
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os
REPO_URL = os.environ.get('FPUS23_REPO_URL', 'https://github.com/Srinivas-Raghav-VC/MultiFetalOrgan-Detection.git')
REPO_DIR = '/content/fpus23'
if not os.path.isdir(REPO_DIR):
    !git clone $REPO_URL $REPO_DIR
else:
    !git -C $REPO_DIR fetch && git -C $REPO_DIR pull --ff-only
%cd /content/fpus23
os.makedirs('/content/drive/MyDrive/FPUS23_runs', exist_ok=True)


In [ ]:
from pathlib import Path
YOLO_BALANCED = Path('/content/fpus23_project/dataset/fpus23_yolo_balanced/data.yaml')
if not YOLO_BALANCED.exists():
    print('Balanced YAML not found, falling back to original YOLO data.yaml')
    YOLO_BALANCED = Path('/content/fpus23_project/dataset/fpus23_yolo/data.yaml')
assert YOLO_BALANCED.exists(), 'Missing dataset YAML. Prepare dataset first.'
print('Using', YOLO_BALANCED)


In [ ]:
sizes = ['n','s','m']
batches = {'n':16,'s':16,'m':8}
for sz in sizes:
  run = f'fpus23_optimized_{sz}'
  !yolo train data=$YOLO_BALANCED model=yolo11{sz}.pt epochs=100 batch={batches[sz]} imgsz=768 \
       project='/content/drive/MyDrive/FPUS23_runs' name=$run deterministic=True workers=2 \
       cos_lr=True cls=3.0 resume=True save_period=1


In [ ]:
import pandas as pd
from pathlib import Path
BASE = Path('/content/drive/MyDrive/FPUS23_runs')
runs = ['fpus23_optimized_n','fpus23_optimized_s','fpus23_optimized_m']
rows = []
for r in runs:
  p = BASE / r / 'results.csv'
  if p.exists():
    df = pd.read_csv(p).dropna(how='all')
    if len(df):
      last = df.tail(1).squeeze()
      rows.append({'run': r, 'epoch': last.get('epoch'),
                   'mAP50': last.get('metrics/mAP50(B)'),
                   'mAP50-95': last.get('metrics/mAP50-95(B)'),
                   'precision': last.get('metrics/precision(B)'),
                   'recall': last.get('metrics/recall(B)'),
                   'csv': str(p)})
if rows:
  import pandas as pd
  display(pd.DataFrame(rows).sort_values('mAP50-95', ascending=False))
else:
  print('No results found yet. Check your Drive runs folder.')
